In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pip install regex

     |████████████████████████████████| 655kB 2.8MB/s 
  Created wheel for regex: filename=regex-2019.8.19-cp36-cp36m-linux_x86_64.whl size=609251 sha256=d96f1331c9b82ef4fcc02e0774f5b94e9ee650243a4134845e20adf193ccad63
  Stored in directory: /root/.cache/pip/wheels/90/04/07/b5010fb816721eb3d6dd64ed5cc8111ca23f97fdab8619b5be
Successfully built regex


In [0]:
import regex as re
import json
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot  as plt
from keras.models import Sequential
from keras import layers
from keras.callbacks import EarlyStopping
from keras import optimizers
import statistics
import seaborn as sns
from keras import backend as K
from collections import Counter
from sklearn import preprocessing

Using TensorFlow backend.


In [0]:
with open("/content/gdrive/My Drive/Summer Project/tagging_data.json", encoding = "UTF-8") as f:
    telecom_tagging = json.load(f)
with open("/content/gdrive/My Drive/Summer Project/final_counseling_data.json", encoding = "UTF-8") as f:
    counseling = json.load(f)
with open("/content/gdrive/My Drive/Summer Project/answer_DB.json", encoding = "UTF-8") as f:
    answers = json.load(f)

In [0]:
len(answers) == 148224
answer_df = pd.DataFrame(answers).drop_duplicates(subset=['receiptNum'])
answer_df.shape

(145380, 10)

In [0]:
counseling_df = pd.DataFrame(counseling)
counseling_df = counseling_df.drop('receiptDate', axis = 1)
counseling_df = counseling_df.drop_duplicates(subset=['receiptNum'])

In [0]:
tags = list()

for rank in telecom_tagging:
  rank.pop('num', None)
  tags.append(rank)
tags_df = pd.DataFrame(tags).drop_duplicates(subset=['receiptNum'])

In [0]:
tags_df.head(2)

,qKeyword,receiptNum
0,"[개통, 철회, 요청, 전화, 신고, 제도, 안내, 가입, 처리, 위약, 부당, 불...",2019-0480284
1,"[사용, 무료, 이동, 이상, 연락, 입금, 전화]",2019-0479644


In [0]:
print(answer_df.shape)
print(counseling_df.shape)
print(tags_df.shape)

(145380, 10)
(145380, 10)
(145582, 2)


In [0]:
counter = Counter()
counter.update(tags_df['receiptNum'])

In [0]:
exclude = list()
for k,v in counter.items():
    if v >= 2:
        exclude.append(k)

In [0]:
len(exclude)

0

In [0]:
final = pd.merge(counseling_df, tags_df, how = 'left', on = 'receiptNum')
final = pd.merge(final, answer_df, how = 'left', on = 'receiptNum')
final = final.drop(columns=['num', 'answer_y'], axis=1)
final = final.rename(index = str, columns = {"answer_x":'answer'})

In [0]:
bigCatList = list(set(final.gooboon))
bigCatList

['보건·위생서비스',
 '문화용품',
 '정보통신서비스',
 '가사용품',
 '가구',
 '의료서비스',
 '식료품·기호품',
 '식생활기기',
 '법률·행정서비스',
 '광열·수도',
 '문화·오락서비스',
 '의류·섬유신변용품',
 '금융',
 '정보통신기기',
 '스포츠·레저·취미용품',
 '주방용품·설비',
 '도서·음반',
 '기계류기타물품',
 '기타서비스',
 '운수·보관·관리서비스',
 '교육서비스',
 '세탁서비스',
 '보건·위생용품',
 '차량·승용물',
 '농·수·축산용품',
 '보험',
 '토지·건물·설비']

In [0]:
targetList = ['add_paper', 'nego_able', 'outer']
dataList = list()
keyList = list()
for name in bigCatList:
  foo = dict()
  foo['bigCat'] = name
  df = final[final.gooboon == name]
  foo['dataframe'] = df
  dataList.append(foo)
  keys = list()
  for i in df['qKeyword']:
    for k in i:
      if k not in keys:
        keys.append(k)
      else:
        continue
  keyList.append(keys)
  print("Done making keyList for", name)

Done making keyList for 보건·위생서비스
Done making keyList for 문화용품
Done making keyList for 정보통신서비스
Done making keyList for 가사용품
Done making keyList for 가구
Done making keyList for 의료서비스
Done making keyList for 식료품·기호품
Done making keyList for 식생활기기
Done making keyList for 법률·행정서비스
Done making keyList for 광열·수도
Done making keyList for 문화·오락서비스
Done making keyList for 의류·섬유신변용품
Done making keyList for 금융
Done making keyList for 정보통신기기
Done making keyList for 스포츠·레저·취미용품
Done making keyList for 주방용품·설비
Done making keyList for 도서·음반
Done making keyList for 기계류기타물품
Done making keyList for 기타서비스
Done making keyList for 운수·보관·관리서비스
Done making keyList for 교육서비스
Done making keyList for 세탁서비스
Done making keyList for 보건·위생용품
Done making keyList for 차량·승용물
Done making keyList for 농·수·축산용품
Done making keyList for 보험
Done making keyList for 토지·건물·설비


In [0]:
tag_list = list()
for i in range(len(bigCatList)):
  foo = dict()
  foo[bigCatList[i]] = keyList[i]
  tag_list.append(foo)

In [0]:
###### this version is the real version. But it is not used in this notebook.
# tag_list = dict()
# for i in range(len(bigCatList)):
#   tag_list[bigCatList[i]] = keyList[i]
#   tag_list.append(foo)
# with open('/content/gdrive/My Drive/Summer Project/newModels/tag_list.json', 'w')  as f:
#   json.dump(tag_list, f)

In [0]:
tag_list['식생활기기'][:10]

In [0]:
dataList[1]['dataframe'].head(1)

In [0]:
dataList[0].keys()

In [0]:
len(keyList[0]), len(keyList[1]), len(keyList[2])

In [0]:
# newDataList = list()

# for d in range(len(dataList)):
#   df = dataList[d]['dataframe']
#   kl = keyList[d]
#   new = list()
#   for k in range(len(df.qKeyword)):
#     new.append([1 if i in df.qKeyword[k] else 0 for i in kl])
#   newdf = pd.DataFrame(new, columns=kl)
#   newDataList.append(newdf)
#   print(dataList[d]['bigCat'])

In [0]:
# for d in range(len(newDataList)):
#   newDataList[d].to_json('/content/gdrive/My Drive/Summer Project/newData/'+bigCatList[d]+'.json')  

In [0]:
newDataList = list()
# finalDataList = dict()
for i in bigCatList:
  with open('/content/gdrive/My Drive/Summer Project/newData/'+i+'.json', encoding = 'utf-8') as file:
    t = json.load(file)
  t = pd.DataFrame(t)
#   finalDataList[i]=t
  newDataList.append(t)

In [0]:
for i in range(len(keyList)):
  if newDataList[i].shape[1] != len(keyList[i]):
    print(i)

In [0]:
for i in range(len(bigCatList)):
  if dataList[i]['dataframe'].shape[0] != newDataList[i].shape[0]:
    print(i)

In [0]:
finalDataList = list()
for i in range(len(bigCatList)):
  finalDataList.append(pd.concat([dataList[i]['dataframe'].reset_index(drop=True), newDataList[i].reset_index(drop=True)], axis=1))

In [0]:
i = 1
finalDataList[i].shape, dataList[i]['dataframe'].shape, newDataList[i].shape, len(keyList[i])

((13342, 426), (13342, 18), (13342, 408), 408)

In [0]:
def pipe(data, keyList_new, target):
    x_train, x_test, y_train, y_test = train_test_split(data.loc[:, keyList_new], 
                                                        data.loc[:, target] ,
                                                        random_state = 123)

    y_train = [1 if i=='yes' else 0 for i in y_train]
    y_test = [1 if i=='yes' else 0 for i in y_test]
    
    x_train = x_train.to_dense()
    x_test = x_test.to_dense()
    
    input_dim = x_train.shape[1]
    
    model = Sequential()
    optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
    
    if data.shape[0] < 5000:
        model.add(layers.Dense(5, input_dim = input_dim, activation = 'relu'))
        model.add(layers.Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', 'binary_crossentropy'])
        es = EarlyStopping(monitor='val_loss', mode='min')
    else:
        model.add(layers.Dense(50, input_dim = input_dim))
        model.add(layers.Dense(10, input_dim = input_dim))
        model.add(layers.Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', 'binary_crossentropy'])
        es = EarlyStopping(monitor='val_loss', mode='min')        
    
    history = model.fit(x_train, y_train, epochs = 100, verbose = False, 
                        validation_data = (x_test, y_test), batch_size=10, callbacks=[es])
    
    return x_train, x_test, y_train, y_test, model, history 

In [0]:
finalDataList[0].shape

(1390, 246)

In [0]:
targetList = ['add_paper', 'nego_able', 'outer', 'goojae', 'jojung']

In [0]:
finalDataList[i]['outer'].dtypes

CategoricalDtype(categories=['', 'counsel.seoul.go.kr', 'cyber.seoul.go.kr',
                  'economy.seoul.go.kr', 'edc.me.go.kr', 'housing.seoul.go.kr',
                  'sobi.jeonbuk.go.kr', 'www.acrc.go.kr', 'www.adc.go.kr',
                  'www.assembly.go.kr', 'www.ccn.go.kr', 'www.ctrc.go.kr',
                  'www.epost.go.kr', 'www.ftc.go.kr', 'www.kats.go.kr',
                  'www.mltm.go.kr', 'www.moleg.go.kr', 'www.molit.go.kr',
                  'www.netan.go.kr', 'www.nier.go.kr', 'www.nssc.go.kr',
                  'www.nts.go.kr', 'www.scourt.go.kr'],
                 ordered=False)

In [0]:
from collections import Counter
ans = list()
counter = Counter()
for i in range(len(finalDataList)):
  for n in range(len(finalDataList[i])):
    if 'www' not in finalDataList[i].loc[n, 'outer']:
      finalDataList[i].loc[n, 'outer'] = ''
      
  finalDataList[i]['outer'] = finalDataList[i]['outer'].astype('category')
  for n in range(len(finalDataList[i])):
    foo = finalDataList[i].loc[n, 'outer']
    if foo not in ans:
      ans.append(foo)
    else:
      continue
  print("Done for ", i)

In [0]:
#### make a dictionary for the 
outerDict = dict()
for i in range(len(ans)):
  outerDict[i] = ans[i]

LabelEncoder()

In [0]:
# dataList, keyList, targetList, newDataList
for number in range(len(finalDataList)):
  for target in range(2):
    x_train, x_test, y_train, y_test, model, history = \
            pipe(finalDataList[number], keyList[number], targetList[target])
    model.save('/content/gdrive/My Drive/Summer Project/newModels/' + bigCatList[number]+'_'+targetList[target]+'.h5')
    loss, accuracy, binary_crossentropy = model.evaluate(x_test, y_test, verbose=False)
    
    print(bigCatList[number] + targetList[target] + '/// Length:' + str(x_train.shape) + '/// Accuracy: ' + str(accuracy))

In [0]:
targetList = ['add_paper', 'nego_able', 'outer', 'goojae', 'jojung']
# dataList, keyList, targetList, newDataList
for number in range(len(finalDataList)):
  for target in range(3, 5):
    x_train, x_test, y_train, y_test, model, history = \
            pipe(finalDataList[number], keyList[number], targetList[target])
    model.save('/content/gdrive/My Drive/Summer Project/newModels/' + bigCatList[number]+'_'+targetList[target]+'.h5')
    loss, accuracy, binary_crossentropy = model.evaluate(x_test, y_test, verbose=False)
    
    print(bigCatList[number] + targetList[target] + '/// Length:' + str(x_train.shape) + '/// Accuracy: ' + str(accuracy))

In [0]:
def pipe2(data, keyList_new, target):
    x_train, x_test, y_train, y_test = train_test_split(data.loc[:, keyList_new], 
                                                        data.loc[:, target] ,
                                                        random_state = 123)
    le = preprocessing.LabelEncoder()
    le.fit(data.loc[:, target])
    print(le.classes_)
    y_train = le.transform(y_train)
    y_test = le.transform(y_test)

    x_train = x_train.to_dense()
    x_test = x_test.to_dense()
    
    input_dim = x_train.shape[1]
    
    model = Sequential()
    optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
    
    if data.shape[0] < 5000:
        model.add(layers.Dense(5, input_dim = input_dim, activation = 'relu'))
        model.add(layers.Dense(1, activation='softmax'))
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', 'binary_crossentropy'])
        es = EarlyStopping(monitor='val_loss', mode='min')
    else:
        model.add(layers.Dense(50, input_dim = input_dim))
        model.add(layers.Dense(10, input_dim = input_dim))
        model.add(layers.Dense(1, activation='softmax'))
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', 'binary_crossentropy'])
        es = EarlyStopping(monitor='val_loss', mode='min')        
    
    history = model.fit(x_train, y_train, epochs = 100, verbose = False, 
                        validation_data = (x_test, y_test), batch_size=10, callbacks=[es])
    
    return x_train, x_test, y_train, y_test, model, history, le

In [0]:
targetList = ['add_paper', 'nego_able', 'outer', 'goojae', 'jojung']
# dataList, keyList, targetList, newDataList
for number in range(len(finalDataList)):
  for target in range(2,3):
    x_train, x_test, y_train, y_test, model, history, le = \
            pipe2(finalDataList[number], keyList[number], targetList[target])
    model.save('/content/gdrive/My Drive/Summer Project/newModels/' + bigCatList[number]+'_'+targetList[target]+'.h5')
    loss, accuracy, binary_crossentropy = model.evaluate(x_test, y_test, verbose=False)
    
    print(bigCatList[number] + targetList[target] + '/// Length:' + str(x_train.shape) + '/// Accuracy: ' + str(accuracy))

['' 'www.customs.go.kr' 'www.epost.go.kr' 'www.fsc.go.kr' 'www.ftc.go.kr'
 'www.nts.go.kr' 'www.police.go.kr' 'www.scourt.go.kr']
보건·위생서비스outer/// Length:(1042, 228)/// Accuracy: 0.0
['' 'www.acrc.go.kr' 'www.ccn.go.kr' 'www.ctrc.go.kr' 'www.customs.go.kr'
 'www.epost.go.kr' 'www.ftc.go.kr' 'www.hometax.go.kr' 'www.kats.go.kr'
 'www.kcc.go.kr' 'www.kipo.go.kr' 'www.moleg.go.kr' 'www.motie.go.kr'
 'www.netan.go.kr' 'www.npa.go.kr' 'www.open.go.kr' 'www.police.go.kr']
문화용품outer/// Length:(4070, 373)/// Accuracy: 0.0
['' 'www.acrc.go.kr' 'www.ccn.go.kr' 'www.ctrc.go.kr'
 'www.cybercrime.go.kr' 'www.donotcall.go.kr' 'www.epost.go.kr'
 'www.etrc.go.kr' 'www.ftc.go.kr' 'www.kbc.go.kr' 'www.kcc.go.kr'
 'www.koreapost.go.kr' 'www.mcst.go.kr' 'www.mke.go.kr' 'www.mltm.go.kr'
 'www.moel.go.kr' 'www.msip.go.kr' 'www.msit.go.kr' 'www.netan.go.kr'
 'www.npa.go.kr' 'www.nta.go.kr' 'www.nts.go.kr' 'www.police.go.kr'
 'www.scourt.go.kr' 'www.wiseuser.go.kr']


KeyboardInterrupt: ignored

In [0]:
from keras.models import load_model
an = '의류·섬유신변용품'
model = load_model('/content/gdrive/My Drive/Summer Project/newModels/' + an + '_outer.h5')
with open('/content/gdrive/My Drive/Summer Project/newModels/tag_list.json', encoding = 'utf-8') as f:
  taglist = json.load(f)
a = ['문화', '발톱', '아픔', '사기']
tt = pd.DataFrame([[1 if _ in a else 0 for _ in taglist[an]]])    
model.predict(tt)  

array([[1.]], dtype=float32)

In [0]:
# finalDataList, tag_list, bigCatList
len(finalDataList)

In [0]:
tag_list.keys()

In [0]:
finalDataList.keys()

In [0]:
keys = list()
track = 0
for i in final.qKeyword:
  track += 1
  if track%10000 == 0:
    print(track)
  for k in i:
    if k not in keys:
      keys.append(k)
    else:
      continue  

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000


In [0]:
len(keys)

2481

In [0]:
dat = list()
track = 0
for i in final.qKeyword:
  track += 1
  if track%10000 == 0:
    print(track)
  dat.append([1 if _ in i else 0 for _ in keys])
dat = pd.DataFrame(dat)
dat.columns = keys

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000


In [0]:
temp = final.copy()

In [0]:
final.shape, dat.shape, pd.merge(final.reset_index(drop=True), dat.reset_index(drop=True), left_index=True, right_index=True).shape

((145380, 18), (145380, 2481), (145380, 2499))

In [0]:
final= pd.merge(temp.reset_index(drop=True), dat.reset_index(drop=True), left_index=True, right_index=True)

In [0]:
for target in range(len(targetList)):
  x_train, x_test, y_train, y_test, model, history = pipe(final, keys, targetList[target])
  loss, accuracy, binary_crossentropy = model.evaluate(x_test, y_test, verbose=False)
  print(targetList[target] + '/// Length:' + str(x_train.shape) + '/// Accuracy: ' + str(accuracy))

add_paper/// Length:(109035, 2481)/// Accuracy: 0.6641353693817256
nego_able/// Length:(109035, 2481)/// Accuracy: 0.7651671481568739
outer/// Length:(109035, 2481)/// Accuracy: 1.0
goojae/// Length:(109035, 2481)/// Accuracy: 0.7741092309775978
jojung/// Length:(109035, 2481)/// Accuracy: 0.778621543534505


In [0]:
max([len(_) for _ in final.qKeyword])

34

,0
0,123


In [0]:
temp.shape

(145380, 18)

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer

In [0]:
### test train split
# x_train, x_test, y_train, y_test = train_test_split(temp.)

In [0]:
num_labels = 20
vocab_size = 15000
batch_size = 100

In [0]:
vocab_size = 15000
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_posts[0])

NameError: ignored

In [0]:
x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')
x_train